In [1]:
!pip install transformers

In [2]:
img_dir = '/kaggle/input/col774-2022/images/images/'
train_path_x ='/kaggle/input/col774-2022/train_x.csv'
train_path_y ='/kaggle/input/col774-2022/train_y.csv'
test_path_x = '/kaggle/input/col774-2022/non_comp_test_x.csv'
test_path_y = '/kaggle/input/col774-2022/non_comp_test_y.csv'
final_test_path_x='/kaggle/input/col774-2022/comp_test_x.csv'

In [3]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from transformers import BertModel, BertConfig, BertTokenizer
from torch.utils.data import DataLoader, Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AdamW 

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import numpy as np
import copy
# import torch
from torch.utils.data import Dataset
# from torchvision import datasets
import torchvision
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image
import torchvision.models as models
# from torch.autograd import Variable

In [5]:
!pip install -q efficientnet_pytorch

In [6]:
# pretrained models
from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

In [4]:
!CUBLAS_WORKSPACE_CONFIG=:4096:2
torch.backends.cudnn.deterministic = True
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
pretrained_model_name = 'bert-base-uncased'

In [9]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
def get_data(path_x, path_y):
    title_info = pd.read_csv(path_x).set_index('Id')
    img_labels = pd.read_csv(path_y).set_index('Id')
    x, y = [], []
    for i in range(len(img_labels['Genre'])):
        title = title_info['Title'][i]
        img_label = img_labels['Genre'][i]
        x.append(title)
        y.append(int(img_label))
    return x, y

In [11]:
# train_x, train_y = get_data(train_path_x, train_path_y)
test_x, test_y   = get_data(test_path_x, test_path_y)

In [12]:
def get_data_comp(path_x):
    title_info = pd.read_csv(path_x).set_index('Id')
    x, y = [], []
    for i in range(len(title_info['Title'])):
        title = title_info['Title'][i]
        x.append(title)
        y.append(0)
    return x, y

In [13]:
comp_test_x, comp_test_y   = get_data_comp(final_test_path_x)

In [14]:
len(comp_test_x)

11514

In [13]:
# train_encodings = tokenizer(train_x, truncation=True, padding=True, max_length=50)
num_labels = 30
test_encodings  = tokenizer(test_x, truncation=True, padding=True, max_length=50)

In [15]:
num_labels = 30
comp_test_encodings  = tokenizer(comp_test_x, truncation=True, padding=True, max_length=50)

In [16]:
# create the dataloaders
class TitleHeadingDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item



In [17]:
class BookCoverImageDataset(Dataset):
    def __init__(self, train_path_x, train_path_y, img_dir,transform=None,typ='train'):
        
        self.img_names  = pd.read_csv(train_path_x)
        self.img_names  = self.img_names.set_index('Id')
        
        self.img_dir = img_dir
        self.xtransform=transform
        self.typ=typ#labels related var change
        if(self.typ=='train'):
            self.img_labels = pd.read_csv(train_path_y)
            self.img_labels = self.img_labels.set_index('Id')
        # self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])    

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):        
        img_path = os.path.join(self.img_dir, self.img_names['Cover_image_name'][idx])
        image = Image.open(img_path)
#         image = image/255.0
        # image = self.normalize(image)
        if (self.xtransform):
            image =self.xtransform(image)
        else:
            print('other..')
            transform = transforms.Compose([transforms.ToTensor()])
            image = transform(image)
            image =image/255.0
        if (self.typ=='train'):
            label = self.img_labels['Genre'][idx]
        else:
            label=0
        # features = feature_extractor(image)
        label=torch.tensor(label)
        return image, label

In [18]:
# Preprocess image
model_name = 'efficientnet-b3'
image_size = EfficientNet.get_image_size(model_name) # 300
tfms = transforms.Compose([transforms.Resize(image_size), transforms.CenterCrop(image_size), 
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
# img = tfms(img).unsqueeze(0)

In [ ]:
#rnn
# train_dataset = TitleHeadingDataset(train_encodings, train_y)
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [17]:
#rnn
test_dataset = TitleHeadingDataset(test_encodings, test_y)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# #cnn
# cnn_training_data = BookCoverImageDataset(train_path_x, train_path_y, img_dir)
# cnn_train_dataloader = DataLoader(cnn_training_data, batch_size=16, shuffle=True)


In [29]:
cnn_test_data = BookCoverImageDataset(test_path_x, test_path_y, img_dir,tfms)
cnn_test_dataloader  = DataLoader(cnn_test_data, batch_size=32, shuffle=False)

In [19]:
#cnn 
#comp_test
fcnn_test_data = BookCoverImageDataset(final_test_path_x, test_path_y, img_dir,tfms,'test')
fcnn_test_dataloader  = DataLoader(fcnn_test_data, batch_size=32, shuffle=False)

In [ ]:
# load the model
load_path = '/kaggle/input/models/Models/BertModel_fine_tuned_v1.pt'
model = BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=num_labels)
model.load_state_dict(torch.load(load_path))
model.to(device)
model.eval()

In [21]:
comp_test_dataset = TitleHeadingDataset(comp_test_encodings, comp_test_y)
comp_test_dataloader = DataLoader(comp_test_dataset, batch_size=64, shuffle=False)

In [ ]:
# load the model_cnn
load_pt = '/kaggle/input/models/model_cnn.pth'
model_cnn = EfficientNet.from_pretrained('efficientnet-b3')
num_classes = 30
model_cnn._fc=nn.Linear(in_features=1536, out_features=num_classes, bias=True)
model_cnn.load_state_dict(torch.load(load_pt))
model_cnn.to(device)
model_cnn.eval()

In [ ]:
# def train(train_dataloader, optimizer, given_model, epochs=100, model_name=pretrained_model_name, num_classes=30, verbose=True): # returns the trained model
#     device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#     model = given_model
#     model.to(device)
#     model.train()

#     optim = optimizer(model.parameters(), lr=5e-5)
    
#     for epoch in range(epochs):        
#         batch_cnt = 0
#         for batch in train_dataloader:
#             batch_cnt += 1

#             optim.zero_grad()            
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs[0]
#             loss.backward()
#             optim.step()

#             if verbose and batch_cnt%100:
#                 print('loss = {}'.format(loss.item()))
    
#     return model

In [ ]:
# model = train(train_dataloader, optimizer=AdamW, given_model=model, epochs=10)

In [18]:
# # save the model
# save_path = '/content/drive/MyDrive/ml_a4/BertModel_fine_tuned_v2.pt'
# torch.save(model.state_dict(), save_path)

In [15]:
# def accuracy(dataloader, model):     
#     softmax = nn.Softmax(dim=1)        
#     correct = 0.0
#     with torch.no_grad():    
#         for batch in dataloader:        
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             output = model(input_ids, attention_mask=attention_mask, labels=labels)            
#             # apply softmax to output of model
#             preds = softmax(output['logits'])                        
            
#             # move logits to cpu
#             preds = preds.detach().cpu().numpy()
#             label_ids = labels.to('cpu').numpy()
            
#             # get correct classifications
#             correct += np.sum(label_ids == preds.argmax(1))            

#     return correct/len(dataloader.dataset)

In [16]:
# # train set accuracy
# print(accuracy(train_dataloader, model))
# # print(accuracy(test_dataloader, model))

0.9835380116959064


In [17]:
# # test set accuracy
# print(accuracy(test_dataloader, model))

0.5985964912280701


In [23]:
def get_predictions_rnn(dataloader, model):     
    softmax = nn.Softmax(dim=1)        
    outputs=[]
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    with torch.no_grad():    
        for batch in dataloader:        
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            output = model(input_ids, attention_mask=attention_mask)#, labels=labels)         
            # apply softmax to output of model
            preds = softmax(output['logits'])                        
            # move logits to cpu
            preds = preds.detach().cpu().numpy()
            # get correct classifications
            outputs=outputs+list(preds)            

    return outputs

In [24]:
def get_predictions_cnn(dataloader, model):
    softmax = nn.Softmax(dim=1)
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    outputs=[]
    with torch.no_grad():    
        for x,y in dataloader:
            train_x, train_y = x, y#batch[0], batch[1]
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            logits = model(train_x)        
#             _, preds = torch.max(output.data, 1)
            preds=softmax(logits)
            preds = preds.detach().cpu().numpy()
            # get correct classifications
            outputs=outputs+list(preds)            

    return outputs

In [23]:
p_rnn=get_predictions_rnn(test_dataloader,model)

In [25]:
p_rnn=np.array(p_rnn)
p_rnn.shape

(5700, 30)

In [30]:
p_cnn=get_predictions_cnn(cnn_test_dataloader,model_cnn)

In [31]:
p_cnn=np.array(p_cnn)
p_cnn.shape

(5700, 30)

In [25]:
pf_rnn=get_predictions_rnn(comp_test_dataloader,model)


In [26]:
pf_cnn=get_predictions_cnn(fcnn_test_dataloader,model_cnn)

In [27]:
pf_rnn=np.array(pf_rnn)
print(pf_rnn.shape)


(11514, 30)


In [28]:
pf_cnn=np.array(pf_cnn)
print(pf_cnn.shape)

(11514, 30)


In [29]:
#final 
pf = np.hstack((pf_rnn,pf_cnn))
pf.shape

(11514, 60)

In [30]:
np.save ('/kaggle/working/final_px.npy',pf)

In [35]:
pf=np.load('/kaggle/input/models/final_px_1.npy')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/models/final_px_1.npy'

In [32]:
p=np.hstack((p_rnn,p_cnn))
yp=np.array(test_y)

In [64]:
np.save ('/kaggle/working/val_px_1.npy',p)
np.save ('/kaggle/working/val_y_1.npy',yp)

In [11]:
p=np.load('/kaggle/input/models/val_px_1.npy')
yp=np.load('/kaggle/input/models/val_y_1.npy')

In [7]:
#actual train data
pt=np.load('/kaggle/input/models/train_px_1.npy')
ypt=np.load('/kaggle/input/models/train_y_1.npy')

In [8]:
pt.shape

(34200, 60)

In [96]:
print(p.shape)
yp.shape

(5700, 60)


(5700,)

In [35]:
classes=30
gamma=0.001
k=5

In [5]:
from time import time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
#gaussian_libsvm
t1=time()
svm_main =SVC(C=1.0,kernel ='rbf',gamma =0.001,verbose =True,tol=1e-5,decision_function_shape='ovo')#break_ties
clf = make_pipeline(StandardScaler(), svm_main)
classifier=clf.fit(p,yp)
t=time()-t1

In [38]:
#details-
classes=svm_main.classes_
print("classes-")
print(classes)
print('fit_status',svm_main.fit_status_)
nf =svm_main.n_features_in_
print('n_features_in_',nf)



classes-
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
fit_status 0
n_features_in_ 60


In [39]:
#train acc-val
acc=clf.score(p,yp)
print("training time",t)
print("test accuracy",acc*100)
#confusion matrix

training time 2.1171562671661377
test accuracy 61.631578947368425


In [ ]:
#loaded p,yp data

In [93]:
from sklearn.model_selection import GridSearchCV

In [132]:
#GridSearch
#5-fold cross validation-
svm_main =SVC(decision_function_shape='ovr')#break_ties
cross_k=5
cvals =[1e-5,1e-3,1,5,10]
gammas = [1,0.1,0.01,0.001]
t1=time()
params_grid={'C':cvals,'gamma':gammas,'kernel': ['rbf'],'tol':[1e-5]}
clf = GridSearchCV(svm_main,params_grid,cv =cross_k)

clf.fit(p,yp)
t=time()-t1
#plots

In [133]:
print('best_estimator_')
print(clf.best_estimator_)

print('best_score_')
print(clf.best_score_)

print('best_params_')
print(clf.best_params_)

print('best_index_',clf.best_index_)

print('scorer_',clf.scorer_)
#train acc-val

#train acc-val
acc=clf.score(p,yp)
print("training time",t)
print(" accuracy",acc*100)
#confusion matrix

best_estimator_
SVC(C=5, gamma=0.1, tol=1e-05)
best_score_
0.6110526315789474
best_params_
{'C': 5, 'gamma': 0.1, 'kernel': 'rbf', 'tol': 1e-05}
best_index_ 13
scorer_ <function _passthrough_scorer at 0x7f47acfdd290>
training time 224.20675015449524
 accuracy 62.59649122807017


In [134]:
acc=clf.score(pt,ypt)
print(" train accuracy",acc*100)

 train accuracy 9.845029239766081


In [ ]:
clf.cv_results_

In [100]:
type(clf.cv_results_)

dict

In [108]:
df= pd.DataFrame(clf.cv_results_)

In [ ]:
groups = df.groupby(by=['param_kernel', 'param_tol'])
groups.get_group(('rbf', 1e-5))

In [19]:
yp.shape

(5700,)

In [22]:
pn =np.vstack((pt,p))
ypn =np.vstack((ypt.reshape(-1,1),yp.reshape(-1,1)))
print(pn.shape)
print(ypn.shape)

(39900, 60)
(39900, 1)


In [24]:
ypn =ypn.reshape(-1)

In [25]:
#best model according to gridsearch
#gaussian_libsvm-ovr?
t1=time()
clf =SVC(C=1.0,kernel ='linear',gamma =1.0,tol=1e-5,decision_function_shape='ovo')#break_ties
clf.fit(pn,ypn)
t=time()-t1

In [37]:
t1=time()
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(max_iter=200, warm_start=True,tol=1e-5)
clf.fit(pt,ypt)
t=time()-t1

In [43]:
clf.partial_fit(p,yp)

SGDClassifier(max_iter=200, tol=1e-05, warm_start=True)

In [41]:
#train acc-val
acc=clf.score(pt,ypt)
print("training time",t)
print(" accuracy",acc*100)

training time 0.9257907867431641
 accuracy 99.9298245614035


In [27]:
#train acc-val
acc=clf.score(pn,ypn)
print("training time",t)
print(" accuracy",acc*100)

training time 9.226608276367188
 accuracy 94.52882205513784


In [44]:
#train acc-tn
acc=clf.score(p,yp)
print(" accuracy",acc*100)

 accuracy 61.6140350877193


In [135]:
clf=clf.best_estimator_

In [157]:
clf

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=5.0, decision_function_shape='ovo', gamma=0.1,
                     tol=1e-05))])

In [170]:
import pickle
filename = '/kaggle/working/final_model_lr_c1g1.sav'
# filename='/kaggle/input/models/final_model_gc.sav'

In [171]:
# save the model
pickle.dump(clf, open(filename, 'wb'))

In [35]:
# load the model 
clf0 = pickle.load(open(filename, 'rb'))


In [36]:
clf0

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(decision_function_shape='ovo', gamma=1.0, kernel='linear',
                     tol=1e-05))])

In [37]:
#train acc-tn
acc=clf0.score(pt,ypt)
print(" accuracy",acc*100)


NameError: name 'pt' is not defined

In [26]:
def get_predictions_rnn_only(dataloader, model):     
    softmax = nn.Softmax(dim=1)        
    outputs=[]
    with torch.no_grad():    
        for batch in dataloader:        
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            output = model(input_ids, attention_mask=attention_mask)#, labels=labels)         
            # apply softmax to output of model
            preds = softmax(output['logits'])                        
            # move logits to cpu
            preds = preds.detach().cpu().numpy()
            # get correct classifications
            outputs=outputs+list(preds.argmax(1).reshape(-1))            

    return outputs
y_outs=get_predictions(comp_test_dataloader,model)

In [38]:
#final -f
yf =clf0.predict(pf)


In [30]:
#final -above
yf =clf.predict(pf)

NameError: name 'pf' is not defined

In [183]:
clf

SVC(decision_function_shape='ovo', gamma=1.0, tol=1e-05)

In [44]:
pf.shape

(11514, 60)

In [68]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(in_features=60, out_features=70),
            nn.ReLU(),
            nn.Linear(in_features=70, out_features=50),
            nn.ReLU(),
            nn.Linear(in_features=50, out_features=30)
            #             nn.ReLU(),
            #             nn.Linear(in_features=30, out_features=30)        
        )

    def forward(self, x):
        return self.network(x)

In [69]:
NeuralNetwork()

NeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=60, out_features=70, bias=True)
    (1): ReLU()
    (2): Linear(in_features=70, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=30, bias=True)
  )
)

In [70]:
load_path="/kaggle/input/models/Neuralnet_v1_1_fn.pt"
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(load_path))
model.eval()
model

NeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=60, out_features=70, bias=True)
    (1): ReLU()
    (2): Linear(in_features=70, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=30, bias=True)
  )
)

In [ ]:
def get_predictions_rnn_only(dataloader, model):     
    softmax = nn.Softmax(dim=1)        
    outputs=[]
    with torch.no_grad():    
        for batch in dataloader:        
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            output = model(input_ids, attention_mask=attention_mask)#, labels=labels)         
            # apply softmax to output of model
            preds = softmax(output['logits'])                        
            # move logits to cpu
            preds = preds.detach().cpu().numpy()
            # get correct classifications
            outputs=outputs+list(preds.argmax(1).reshape(-1))            

    return outputs
y_outs=get_predictions(comp_test_dataloader,model)

In [81]:
def get_preds_from_nn(dataloader, model):     
    device  = "cuda" if torch.cuda.is_available() else "cpu"
    softmax = nn.Softmax(dim=1)        
    correct = 0.0
    outputs = []
    with torch.no_grad():    
        for (X, y) in dataloader:        
            X = X.to(device)
            # y = y.to(device)
            out = model(X)
            preds = softmax(out)
            preds = preds.detach().cpu().numpy()
#             y = y.to('cpu').numpy()
#             correct += np.sum(y == preds.argmax(1))          
            outputs=outputs+list(preds.argmax(1).reshape(-1))            
    return outputs


In [77]:
n =len(pf)
train_y_nn=[0 for i in range(n)]

In [79]:
features_train = torch.tensor(pf)
targets_train  = torch.tensor(train_y_nn)
dataset_train = torch.utils.data.TensorDataset(features_train, targets_train)
final_dataloader = DataLoader(dataset_train, batch_size=100, shuffle=False)

In [82]:
yf=get_preds_from_nn(final_dataloader,model)

In [1]:
len(yf)

NameError: name 'yf' is not defined

In [185]:
y_outs=list(yf)
df =pd.read_csv(final_test_path_x)
dicti={'Id':df['Id'],'Genre':y_outs}
df_out=pd.DataFrame(dicti)


In [186]:
df_out.to_csv('/kaggle/working/final_test_y7.csv',index =False)

In [176]:
df1 = pd.read_csv('/kaggle/working/final_test_y7.csv')

In [178]:
df1.head()

,Id,Genre
0,0,8
1,1,20
2,2,27
3,3,6
4,4,6


In [35]:
df1.head()

,Id,Genre
0,0,18
1,1,20
2,2,27
3,3,6
4,4,6
